<a href="https://colab.research.google.com/github/L1KASA/big-data/blob/main/lab2/6409_%D0%A2%D0%B0%D1%85%D0%BC%D0%B0%D0%B7%D0%BE%D0%B2%D0%B0%D0%90%D0%A8_%D0%9B%D0%A02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Подключаемся к диску
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=698cbe983bb62643dac919b34fc544fe10bfc8fc3df206101fbd1d19a634bb36
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [3]:
# Импортируем необходимые библиотеки
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import max, col, count, expr, desc, sum
from pyspark.sql.functions import desc
from geopy.distance import geodesic
from itertools import combinations
# Создаем сессию Spark
conf = SparkSession.builder \
    .appName("Lab_1") \
    .getOrCreate()
conf

In [4]:
# Загружаем данные trips.csv и stations.csv в DataFrame
trips_df = conf.read.csv("/content/drive/MyDrive/trips.csv", header=True, inferSchema=True)
stations_df = conf.read.csv("/content/drive/MyDrive/stations.csv", header=True, inferSchema=True)

In [5]:
trips_df

DataFrame[id: int, duration: int, start_date: string, start_station_name: string, start_station_id: int, end_date: string, end_station_name: string, end_station_id: int, bike_id: int, subscription_type: string, zip_code: string]

In [6]:
stations_df

DataFrame[id: int, name: string, lat: double, long: double, dock_count: int, city: string, installation_date: string]

In [7]:
# Выведем информацию о количестве записей в каждом DataFrame
print("Количество записей в DataFrame trips_df:", trips_df.count())
print("Количество записей в DataFrame stations_df:", stations_df.count())

# Выведем информацию о схеме данных в каждом DataFrame
print("Схема данных в DataFrame trips_df:")
trips_df.printSchema()

print("Схема данных в DataFrame stations_df:")
stations_df.printSchema()

# Посмотрим на первые несколько строк каждого DataFrame
print("Первые строки DataFrame trips_df:")
trips_df.show(5)

print("Первые строки DataFrame stations_df:")
stations_df.show(5)

# Выведем информацию о количестве уникальных значений в различных полях
print("Количество уникальных значений в поле 'start_station_name':", trips_df.select("start_station_name").distinct().count())
print("Количество уникальных значений в поле 'end_station_name':", trips_df.select("end_station_name").distinct().count())
print("Количество уникальных значений в поле 'bike_id':", trips_df.select("bike_id").distinct().count())

Количество записей в DataFrame trips_df: 669959
Количество записей в DataFrame stations_df: 70
Схема данных в DataFrame trips_df:
root
 |-- id: integer (nullable = true)
 |-- duration: integer (nullable = true)
 |-- start_date: string (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- end_date: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- subscription_type: string (nullable = true)
 |-- zip_code: string (nullable = true)

Схема данных в DataFrame stations_df:
root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- dock_count: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- installation_date: string (nullable = true)

Первые строки DataFrame trips_df:
+----+--------+---------------+----

#Задание 1. Найти велосипед с максимальным временем пробега.

In [8]:
max_duration_bike = trips_df.groupBy("bike_id").max("duration").orderBy(desc("max(duration)")).limit(1)
max_duration_bike.show()

+-------+-------------+
|bike_id|max(duration)|
+-------+-------------+
|    535|     17270400|
+-------+-------------+



#Задание 2. Найти наибольшее геодезическое расстояние между станциями.

In [9]:
# Создаем список кортежей с координатами станций (широта, долгота)
stations_coordinates = stations_df.select("name", "lat", "long").collect()

# Используем itertools.combinations, чтобы создать все возможные комбинации станций
station_combinations = combinations(stations_coordinates, 2)

# Инициализируем переменную для хранения наибольшего расстояния
max_distance = 0
station_names = None

# Проходим по всем комбинациям станций
for combo in station_combinations:
    station1 = combo[0]
    station2 = combo[1]

    # Вычисляем геодезическое расстояние между двумя станциями
    distance = geodesic((station1.lat, station1.long), (station2.lat, station2.long)).kilometers

    # Обновляем максимальное расстояние, если текущее больше
    if distance > max_distance:
        max_distance = distance
        station_names = (station1.name, station2.name)

print("Наибольшее геодезическое расстояние между станциями {} и {}: {:.2f} км".format(station_names[0], station_names[1], max_distance))

Наибольшее геодезическое расстояние между станциями SJSU - San Salvador at 9th и Embarcadero at Sansome: 69.92 км


#Задание 3. Найти путь велосипеда с максимальным временем пробега через станции.

In [10]:
# Находим поездку с максимальным временем
max_duration_trip = trips_df.orderBy(desc("duration")).first()

# Выводим информацию о поездке
print("Путь велосипеда с максимальным временем пробега через станции:")
print("Стартовая станция:", max_duration_trip.start_station_name)
print("Конечная станция:", max_duration_trip.end_station_name)
print("Время поездки (в секундах):", max_duration_trip.duration)

Путь велосипеда с максимальным временем пробега через станции:
Стартовая станция: South Van Ness at Market
Конечная станция: 2nd at Folsom
Время поездки (в секундах): 17270400


#Задание 4. Найти количество велосипедов в системе.

In [11]:
# Используем метод distinct() для получения уникальных идентификаторов велосипедов и подсчитываем их количество
total_bikes_count = trips_df.select("bike_id").distinct().count()

print("Количество велосипедов в системе:", total_bikes_count)

Количество велосипедов в системе: 700


#Задание 5. Найти пользователей потративших на поездки более 3 часов.

In [12]:
# Преобразуем время поездок из секунд в часы, разделив на 3600 (количество секунд в часе)
trips_df_with_hours = trips_df.withColumn("duration_hours", col("duration") / 3600)

# Группируем данные по идентификатору пользователя и суммируем длительность поездок для каждого пользователя
user_total_time = trips_df_with_hours.groupBy("bike_id").agg(sum("duration_hours").alias("total_time"))

# Фильтруем данные, оставляя только те поездки, которые длились более 3 часов
long_trips = user_total_time.filter(col("total_time") > 3)

# Выводим результат
print("Пользователи, потратившие более 3 часов на поездки:")
long_trips.show()

Пользователи, потратившие более 3 часов на поездки:
+-------+------------------+
|bike_id|        total_time|
+-------+------------------+
|    471| 477.4530555555561|
|    496| 466.5466666666665|
|    148| 92.26055555555558|
|    463|478.55444444444436|
|    540| 486.8986111111113|
|    392|497.07666666666677|
|    623| 565.8941666666667|
|    243|            85.405|
|    516| 526.8752777777777|
|     31|          113.3075|
|    580| 287.3283333333329|
|    137| 424.7777777777776|
|    251| 356.3833333333333|
|    451| 470.9927777777779|
|     85| 337.4358333333333|
|    458|457.52222222222264|
|     65|60.256111111111125|
|    588| 74.00416666666668|
|    255|110.10972222222225|
|     53| 62.88583333333334|
+-------+------------------+
only showing top 20 rows



#Завершаем сессию

In [13]:
conf.stop()